In [1]:
import numpy as np
import pandas as pd
from collections import Counter

In [2]:
train_infile = 'handout/small_train.tsv'
test_infile = 'handout/small_test.tsv'
attribute = '0'
train_label_outfile = 'pol_0_train.labels'
test_label_outfile = 'pol_0_test.labels'
metric_outfile = 'pol_0_metrics.txt'

In [3]:
df = pd.read_csv(train_infile, sep='\t')
data_train = df.to_numpy()
train_x = data_train[:, :-1]
train_y = data_train[:, -1]

In [4]:
df

,Anti_satellite_test_ban,Export_south_africa,Party
0,n,y,democrat
1,n,y,republican
2,y,y,democrat
3,y,y,democrat
4,y,y,democrat
5,y,y,democrat
6,y,y,democrat
7,n,y,republican
8,y,y,democrat
9,n,n,republican


In [5]:
data_train

array([['n', 'y', 'democrat'],
       ['n', 'y', 'republican'],
       ['y', 'y', 'democrat'],
       ['y', 'y', 'democrat'],
       ['y', 'y', 'democrat'],
       ['y', 'y', 'democrat'],
       ['y', 'y', 'democrat'],
       ['n', 'y', 'republican'],
       ['y', 'y', 'democrat'],
       ['n', 'n', 'republican'],
       ['y', 'y', 'democrat'],
       ['n', 'y', 'republican'],
       ['y', 'y', 'democrat'],
       ['n', 'n', 'republican'],
       ['n', 'y', 'republican'],
       ['n', 'n', 'republican'],
       ['y', 'y', 'democrat'],
       ['y', 'y', 'democrat'],
       ['y', 'y', 'democrat'],
       ['y', 'y', 'democrat'],
       ['n', 'y', 'democrat'],
       ['y', 'y', 'democrat'],
       ['y', 'n', 'republican'],
       ['n', 'n', 'republican'],
       ['n', 'y', 'republican'],
       ['n', 'y', 'republican'],
       ['n', 'y', 'republican'],
       ['n', 'n', 'republican']], dtype=object)

In [6]:
train_x[train_x[:,0] == 'n']

array([['n', 'y'],
       ['n', 'y'],
       ['n', 'y'],
       ['n', 'n'],
       ['n', 'y'],
       ['n', 'n'],
       ['n', 'y'],
       ['n', 'n'],
       ['n', 'y'],
       ['n', 'n'],
       ['n', 'y'],
       ['n', 'y'],
       ['n', 'y'],
       ['n', 'n']], dtype=object)

In [7]:
train_x

array([['n', 'y'],
       ['n', 'y'],
       ['y', 'y'],
       ['y', 'y'],
       ['y', 'y'],
       ['y', 'y'],
       ['y', 'y'],
       ['n', 'y'],
       ['y', 'y'],
       ['n', 'n'],
       ['y', 'y'],
       ['n', 'y'],
       ['y', 'y'],
       ['n', 'n'],
       ['n', 'y'],
       ['n', 'n'],
       ['y', 'y'],
       ['y', 'y'],
       ['y', 'y'],
       ['y', 'y'],
       ['n', 'y'],
       ['y', 'y'],
       ['y', 'n'],
       ['n', 'n'],
       ['n', 'y'],
       ['n', 'y'],
       ['n', 'y'],
       ['n', 'n']], dtype=object)

In [8]:
np.unique(train_y[train_x[:,0] == "y"])[0]

'democrat'

In [9]:
class DecisionStump:
    def __init__(self, attribute: str):
        self.map = {}
        self.attribute_index = int(attribute)
    '''
    train_x: numpy array
    train_y: numpy array
    '''
    def train(self, train_x, train_y):
        binary_values = np.unique(train_x)
        for i in binary_values:
            self.map[i] = Counter(train_y[train_x[:,self.attribute_index] == i]).most_common(1)[0][0]
    
    '''
    x: list or numpy array
    
    return: list
    '''
    def predict(self, x):
        y = []
        for row in x:
            y.append(self.map[row[self.attribute_index]])
        return y

'''
Y: the true result
Y_hat: the predicted result
'''
def error(Y, Y_hat):
    incorrect_num = 0
    for i in range(len(Y)):
        if Y[i] != Y_hat[i]:
            incorrect_num += 1            
    return incorrect_num / len(Y)

In [10]:
train_infile = 'handout/small_train.tsv'
test_infile = 'handout/small_test.tsv'
attribute = '0'
train_label_outfile = 'pol_0_train.labels'
test_label_outfile = 'pol_0_test.labels'
metric_outfile = 'pol_0_metrics.txt'

In [128]:
def main(train_infile, test_infile, attribute, train_label_outfile, test_label_outfile, metric_outfile):
    train_data = np.genfromtxt(train_infile, delimiter='\t', skip_header=1)
    test_data = np.genfromtxt(test_infile, delimiter='\t', skip_header=1)
    
    train_x = train_data[:,:-1]
    train_y = train_data[:,-1]
    test_x = test_data[:, :-1]
    test_y = test_data[:, -1]
    
    dtree = DecisionStump(attribute)
    
    
    dtree.train(train_x, train_y)
    
    train_y_predict = dtree.predict(train_x)
    test_y_predict = dtree.predict(test_x)
    train_error = error(train_y, train_y_predict)
    test_error = error(test_y, test_y_predict)
    
    # write train and test label to file
    with open(train_label_outfile, "w") as fw:
        for i in train_y_predict:
            fw.writeline(i)
            
    with open(test_label_outfile, "w") as fw:
        for i in test_y_predict:
            fw.writeline(i)
            
    with open(metric_outfile, "w") as fw:
        fw.writeline(f"error(train): {train_error}")
        fw.writeline(f"error(test): {test_error}")

In [22]:
def calc_entropy(y):
    numEntries = len(y)
    labelCounts = Counter(y)
    entropy = 0.0
    for key in labelCounts:
        prob = float(labelCounts[key]) / numEntries
        entropy -= prob * np.log2(prob)
    return entropy

def calc_error(Y, Y_hat):
    return np.sum(Y != Y_hat) / len(Y)

class DecisionTree:
    def __init__(self, max_depth):
        # stem node
        self.max_depth_ = max_depth
        self.children = {} # key is the attribute value, value is the child node
        self.split_index_ = None
        # leaf node
        self.label_ = None
    
    
    def Train(self, X, Y):
        self.split_index_ = self.GetSplitIndex(X, Y)
        
        # convert this node to a leaf node
        if self.max_depth_ <= 0 or self.split_index_ is None:
            self.split_index_ = None
            self.label_ = Counter(Y).most_common(1)[0][0]
            return
        
        # keep splitting
        binary_values = np.unique(X[:, self.split_index_])
        for bv in binary_values:
            self.children[bv] = DecisionTree(self.max_depth_ - 1)
            self.children[bv].Train(X[X[:, self.split_index_] == bv], Y[X[:,self.split_index_] == bv])
    
    def DisplayRecursively(self, depth):
        if self.split_index_ is None:
            print(''.join(['\t' for _ in range(depth)]) + self.label_)
        for val, child in self.children.items():
            print(''.join(['\t' for _ in range(depth)]) + str(
                self.split_index_) + " : " + val)
            child.DisplayRecursively(depth + 1)
            
    def Display(self):
        self.DisplayRecursively(0)
        
    '''
    X : M x N numpy array
    
    return M numpy array
    '''
    def Predict(self, X):
        Y = []
        for x in X:
            Y.append(self.PredictRecursively(x))
            
        return np.array(Y)
        
    '''
    X : N numpy array
    return predict label
    '''
    def PredictRecursively(self, X):
        if self.split_index_ is None:
            return self.label_
        return self.children[X[self.split_index_]].PredictRecursively(X)
    
    '''
    x: M x N numpy array
    y: M numpy array
    
    entropy before split - entropy after split
    return a length N numpy array of information gain of all attributes
    '''
    def InformationGain(self, X, Y):
        M, N = X.shape
        
        old_entropy = calc_entropy(Y)
        IG = [old_entropy for _ in range(N)]
        
        for i in range(N):
            col = X[:, i]
            binary_values = np.unique(col)
            for bv in binary_values:
                Y_hat = Y[col == bv]
                IG[i] -= calc_entropy(Y_hat) * len(Y_hat) / len(Y)
                
        return np.array(IG)
    
    '''
    x: M x N numpy array
    y: M numpy array
    
    return the split index if its mutual information is > 0, else None
    '''
    def GetSplitIndex(self, X, Y):
        info_gain = self.InformationGain(X, Y) # length N
        split_index = info_gain.argmax()
        return  split_index if info_gain[split_index] > 0 else None

In [29]:
dtree = DecisionTree(3)

In [30]:
dtree.Train(train_x, train_y)
Y_train_hat = dtree.Predict(train_x)
error_train = calc_error(train_y, Y_train_hat)
# Y_test_hat = dtree.Predict(test_x)
# errot_test = calc_error(test_y, Y_test_hat)

In [32]:
error_train

0.07142857142857142

In [33]:
dtree.Predict(train_x)

array(['republican', 'republican', 'democrat', 'democrat', 'democrat',
       'democrat', 'democrat', 'republican', 'democrat', 'republican',
       'democrat', 'republican', 'democrat', 'republican', 'republican',
       'republican', 'democrat', 'democrat', 'democrat', 'democrat',
       'republican', 'democrat', 'republican', 'republican', 'republican',
       'republican', 'republican', 'republican'], dtype='<U10')

In [24]:
dtree.Display()

0 : n
	1 : n
		republican
	1 : y
		republican
0 : y
	1 : n
		republican
	1 : y
		democrat
